In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [0]:
spark = SparkSession.builder.appName('b').getOrCreate()

In [0]:
df=spark.read.csv('/FileStore/tables/titanic.csv',header=True,inferSchema=True)

In [0]:
df.columns

Out[7]: ['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [0]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler,OneHotEncoder,VectorIndexer
from pyspark.ml.feature import StringIndexer

final_df = df.select(['Survived',
 'Pclass','Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked']).na.drop()
final_df.show()

+--------+------+------+----+-----+-----+-------+--------+
Survived|Pclass| Sex| Age|SibSp|Parch| Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
 0| 3| male|22.0| 1| 0| 7.25| S|
 1| 1|female|38.0| 1| 0|71.2833| C|
 1| 3|female|26.0| 0| 0| 7.925| S|
 1| 1|female|35.0| 1| 0| 53.1| S|
 0| 3| male|35.0| 0| 0| 8.05| S|
 0| 1| male|54.0| 0| 0|51.8625| S|
 0| 3| male| 2.0| 3| 1| 21.075| S|
 1| 3|female|27.0| 0| 2|11.1333| S|
 1| 2|female|14.0| 1| 0|30.0708| C|
 1| 3|female| 4.0| 1| 1| 16.7| S|
 1| 1|female|58.0| 0| 0| 26.55| S|
 0| 3| male|20.0| 0| 0| 8.05| S|
 0| 3| male|39.0| 1| 5| 31.275| S|
 0| 3|female|14.0| 0| 0| 7.8542| S|
 1| 2|female|55.0| 0| 0| 16.0| S|
 0| 3| male| 2.0| 4| 1| 29.125| Q|
 0| 3|female|31.0| 1| 0| 18.0| S|
 0| 2| male|35.0| 0| 0| 26.0| S|
 1| 2| male|34.0| 0| 0| 13.0| S|
 1| 3|female|15.0| 0| 0| 8.0292| Q|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 20 rows

In [0]:
sex_indexer = StringIndexer(inputCol="Sex", outputCol="Sex_Cat")
sex_encoder = OneHotEncoder(inputCol="Sex_Cat", outputCol="Sex_vec")
E_indexer = StringIndexer(inputCol="Embarked", outputCol="Embarked_Cat")
E_encoder = OneHotEncoder(inputCol="Embarked_Cat", outputCol="Embarked_vec")



In [0]:
assembler = VectorAssembler( inputCols=['Pclass','Sex_vec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked_vec'],outputCol='features')

In [0]:
from pyspark.ml import Pipeline 

log_reg_titaninc = LogisticRegression(featuresCol='features',labelCol='Survived')
pipeline = Pipeline(stages=[sex_indexer, E_indexer,sex_encoder,E_encoder,assembler,log_reg_titaninc])
#output = assembler.transform(final_df)

In [0]:
output.head(1)

Out[43]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_cat=16.0, features=DenseVector([16.0, 6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [0]:
final_data = output.select('features','crew')

In [0]:
train_data,test_data = final_df.randomSplit([0.7,0.3])

In [0]:
lr_model = pipeline.fit(train_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
my_Eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')
results = lr_model.transform(test_data)
results.select('prediction','Survived')
AUC=my_Eval.evaluate(results)

In [0]:
AUC

Out[23]: 0.77406588290071

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
predictions = lr_model.transform(unlabeled_data)

In [0]:
from pyspark.sql.functions import corr

In [0]:
df.select(corr('crew','cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+